In [1]:
import torch
import pickle
import numpy as np
import pandas as pd
import os

from os.path import dirname



root_path = dirname(os.getcwd()) + "/HGNN_NA"

pd.set_option("display.max_columns", None)
data_dir = root_path + "/data/datasets/original/"
data_dir_processed = root_path + "/data/datasets/processed/"
data_dir_graphs = root_path + "/data/datasets/graphs/"

print(root_path, data_dir, data_dir_processed, data_dir_graphs, sep="\n")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"

/home/sebdis/ProcessMining/HGNN/HGNN_NA
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/original/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/processed/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/graphs/


In [2]:
dataset = "BPI20_RequestForPayment"

In [ ]:
categorical_columns = ['org:group', 'resource country', 'org:resource', 'oranization country',
       'org:role', 'Activity', 'impact', 'product']
real_value_columns = ["time:timestamp"]

In [3]:
tab_all = pd.read_csv(data_dir_processed+dataset+"_processed_all.csv")
tab_all.head()

,org:resource,Activity,time:timestamp,org:role,case:Project,case:Task,CaseID,case:OrganizationalEntity,case:Cost Type,case:RequestedAmount,case:Activity,case:RfpNumber
0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,0.000000,EMPLOYEE,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215
1,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,3.737670,SUPERVISOR,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215
2,STAFF MEMBER,Request For Payment REJECTED by MISSING,11.499982,MISSING,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215
3,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,15.337479,EMPLOYEE,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215
4,STAFF MEMBER,Request For Payment APPROVED by PRE_APPROVER,15.337486,PRE_APPROVER,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215


In [5]:
tab_all.columns

Index(['org:resource', 'Activity', 'time:timestamp', 'org:role',
       'case:Project', 'case:Task', 'CaseID', 'case:OrganizationalEntity',
       'case:Cost Type', 'case:RequestedAmount', 'case:Activity',
       'case:RfpNumber'],
      dtype='object')

In [26]:
categorical_columns = ['org:resource', 'Activity', 'org:role',
      'case:Project', 'case:Task', 
       'case:OrganizationalEntity', 'case:Cost Type',
       'case:Activity', 'case:RfpNumber']
real_value_columns = ["time:timestamp", "case:RequestedAmount"]

{'org:resource': ['STAFF MEMBER', 'SYSTEM'],
 'Activity': ['Request For Payment SUBMITTED by EMPLOYEE',
  'Request For Payment FINAL_APPROVED by SUPERVISOR',
  'Request For Payment REJECTED by MISSING',
  'Request For Payment APPROVED by PRE_APPROVER',
  'Request Payment',
  'Payment Handled',
  'Request For Payment REJECTED by SUPERVISOR',
  'Request For Payment REJECTED by EMPLOYEE',
  'Request For Payment APPROVED by SUPERVISOR',
  'Request For Payment FINAL_APPROVED by DIRECTOR',
  'Request For Payment REJECTED by PRE_APPROVER',
  'Request For Payment SAVED by EMPLOYEE',
  'Request For Payment REJECTED by ADMINISTRATION',
  'Request For Payment APPROVED by ADMINISTRATION',
  'Request For Payment APPROVED by BUDGET OWNER',
  'Request For Payment REJECTED by BUDGET OWNER',
  'Request For Payment FOR_APPROVAL by ADMINISTRATION',
  'Request For Payment FINAL_APPROVED by BUDGET OWNER',
  'Request For Payment FOR_APPROVAL by SUPERVISOR'],
 'org:role': ['EMPLOYEE',
  'SUPERVISOR',
  'MISS

In [7]:
import random

torch.manual_seed(0)
torch.cuda.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [8]:
with open(data_dir_graphs + dataset + "_TRAIN_event_prediction_FINAL.pkl", "rb") as f:
    X_train = pickle.load(f)
with open(data_dir_graphs + dataset + "_VALID_event_prediction_FINAL.pkl", "rb") as f:
    X_valid = pickle.load(f)
with open(data_dir_graphs + dataset + "_TEST_event_prediction_FINAL.pkl", "rb") as f:
    X_test = pickle.load(f)

In [9]:

from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import ToUndirected, NormalizeFeatures

transform = ToUndirected()

with torch.no_grad():
        for i in range(len(X_train)):
                X_train[i] = transform(X_train[i])
        for i in range(len(X_valid)):
                X_valid[i] = transform(X_valid[i])
        for i in range(len(X_test)):
                X_test[i] = transform(X_test[i])
    






In [10]:
edge_types = set()
node_types = set()
for i in range(len(X_train)):
    n, edge_type = X_train[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)
for i in range(len(X_valid)):
    n, edge_type = X_valid[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)
for i in range(len(X_test)):
    n, edge_type = X_test[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)



In [11]:
node_types = list(node_types)
edge_types = list(edge_types)

In [12]:
node_types

['org:resource',
 'time:timestamp',
 'case:Cost Type',
 'org:role',
 'Activity',
 'case:OrganizationalEntity',
 'case:RequestedAmount',
 'case:Project',
 'case:RfpNumber',
 'case:Activity',
 'case:Task']

In [13]:
edge_types

[('Activity', 'related_to', 'case:Activity'),
 ('Activity', 'related_to', 'case:Cost Type'),
 ('org:role', 'related_to', 'org:role'),
 ('Activity', 'related_to', 'case:Task'),
 ('case:OrganizationalEntity', 'rev_related_to', 'Activity'),
 ('Activity', 'related_to', 'case:RequestedAmount'),
 ('Activity', 'related_to', 'org:resource'),
 ('case:Project', 'rev_related_to', 'Activity'),
 ('case:RequestedAmount', 'rev_related_to', 'Activity'),
 ('org:role', 'rev_related_to', 'Activity'),
 ('time:timestamp', 'related_to', 'time:timestamp'),
 ('Activity', 'related_to', 'case:Project'),
 ('case:Activity', 'rev_related_to', 'Activity'),
 ('Activity', 'related_to', 'case:RfpNumber'),
 ('org:resource', 'rev_related_to', 'Activity'),
 ('time:timestamp', 'rev_related_to', 'Activity'),
 ('Activity', 'related_to', 'case:OrganizationalEntity'),
 ('case:Cost Type', 'rev_related_to', 'Activity'),
 ('Activity', 'followed_by', 'Activity'),
 ('case:RfpNumber', 'rev_related_to', 'Activity'),
 ('org:resource'

In [14]:
def get_weights(load, key):
    weights = []
    
    cl_train = [0 for _ in tab_all[key].unique()]
    
    print(cl_train)
    
    for i,x in enumerate(load):

        
        classes = x.y[key]

        # print(classes)
        
        for c in list(classes):
            try:
                cl_train[c] +=1
            except KeyError:
                cl_train[c] = 1
    s = sum(cl_train)
    
    print(cl_train)
    
    weights = [s/x if x > 0 else 0 for x in cl_train]

    # weights = [0.7,0.7,1,0.7,0.7,0.7,0.7,0.7,0.7,0.7]
    weights = torch.tensor(weights, device=device)
    print(weights)
    return weights
    

In [15]:
edge_features_dims = {}

for i in range(len(X_train)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]
for i in range(len(X_valid)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]
for i in range(len(X_test)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]

edge_features_dims

{('org:resource', 'related_to', 'org:resource'): 2,
 ('Activity', 'followed_by', 'Activity'): 2,
 ('time:timestamp', 'related_to', 'time:timestamp'): 2,
 ('org:role', 'related_to', 'org:role'): 2}

In [16]:
act_weights = get_weights(DataLoader(X_train, batch_size=1024, shuffle=False), "Activity")
# res_roles_weights = get_weights(DataLoader(X_train, batch_size=1024, shuffle=False), "org:resource:role")


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3348, 228, 897, 42, 14, 0, 26, 2778, 1, 0, 31, 19, 534, 66, 2, 82, 1, 341, 3348]
tensor([3.5119e+00, 5.1570e+01, 1.3108e+01, 2.7995e+02, 8.3986e+02, 0.0000e+00,
        4.5223e+02, 4.2325e+00, 1.1758e+04, 0.0000e+00, 3.7929e+02, 6.1884e+02,
        2.2019e+01, 1.7815e+02, 5.8790e+03, 1.4339e+02, 1.1758e+04, 3.4481e+01,
        3.5119e+00], device='cuda:0')


## Hyperopt

In [17]:
from ax.service.managed_loop import optimize

In [18]:
from torch_geometric.nn import (
    HeteroConv,
    global_mean_pool,
    GATv2Conv
)
from torch.nn import (
    ModuleList,
    Module,
    Linear
  )
from typing_extensions import Self

In [19]:
class HGNN(Module):

    def __init__(self, output_cat, output_real,nodes_relations, relations_with_features, parameters) -> Self:  # type: ignore
        super().__init__()

        # List of convolutional layers
        
        hid = parameters["hid"]
        layers = parameters["layers"]
        aggregation = parameters["aggregation"]
        n_heads = parameters["heads"]
        
        self.output_cat = output_cat
        self.output_real = output_real
        
        self.convs = ModuleList()
        for _ in range(layers):
            conv = HeteroConv(
                {
                    relation: (
                        GATv2Conv((-1,-1), add_self_loops=False, out_channels=hid, heads=n_heads, concat=False)
                        if relation not in relations_with_features
                        else GATv2Conv((-1,-1), add_self_loops=False, out_channels=hid,heads=n_heads, edge_dim=relations_with_features[relation], concat=False)
                    )
                    for relation in nodes_relations
                },
                aggr=aggregation,
            )

            self.convs.append(conv)

        self.FC = {}
        
        for k in output_cat:
            self.FC[k] = Linear(hid, output_cat[k], device=device)
        for k in output_real:
            self.FC[k] = Linear(hid, 1, device=device)
        
    
        
        

    def forward(self, batch):

        for i in range(len(self.convs)):
            batch.x_dict = self.convs[i]( 
                batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict
            )

            batch.x_dict = {key: x.relu() for key, x in batch.x_dict.items()}


        output = {}
        
        for k in self.output_cat:
            output[k] = global_mean_pool(batch.x_dict[k], batch[k].batch)
            output[k] = self.FC[k](output[k])
        for k in self.output_real:
            output[k] = global_mean_pool(batch.x_dict[k], batch[k].batch)
            output[k] = self.FC[k](output[k]).reshape(1,-1)[0]
            

        return output

In [20]:
from torcheval.metrics.functional import multiclass_accuracy, multiclass_f1_score

In [21]:
import torch.nn as nn

In [22]:
import time

In [23]:
from copy import deepcopy


def train_hgnn(config, output_cat, output_real, epochs=20):
    print(config)

    net = HGNN(
        parameters=config,
        output_cat=output_cat,
        output_real=output_real,
        nodes_relations=edge_types,
        relations_with_features=edge_features_dims,
    )
    net = net.to(device)

    losses = {}

    for k in output_cat:
        losses[k] = (
            nn.CrossEntropyLoss()
            if k != "Activity"
            else nn.CrossEntropyLoss(act_weights)
        )
    for k in output_real:
        losses[k] = nn.L1Loss()

    train_loader = DataLoader(X_train, batch_size=config["batch_size"], shuffle=True)
    valid_loader = DataLoader(X_valid, batch_size=config["batch_size"], shuffle=True)

    optimizer = torch.optim.Adam(net.parameters(), lr=config["lr"])

    best_model = None
    best_loss = 0
    patience = 5
    pat_count = 0

    torch.cuda.empty_cache()

    for epoch in range(0, epochs):
        start_time = time.time()

        print(f"Epoch: {epoch}\n")

        net.train()
        for _, x in enumerate(train_loader):
            x = x.to(device)

            labels = x.y

            optimizer.zero_grad()
            outputs = net(x)

            losses_step = {k: losses[k](outputs[k], labels[k]) for k in losses}

            total_loss = 0
            for k in losses_step:
                total_loss += losses_step[k]

            total_loss.backward()
            optimizer.step()

        predictions_categorical = {k: [] for k in output_cat}
        target_categorical = {k: [] for k in output_cat}

        avg_MAE = {k: [] for k in output_real}

        running_total_loss = []

        net.eval()
        with torch.no_grad():
            for i, x in enumerate(valid_loader):
                x = x.to(device)

                labels = x.y

                outputs = net(x)

                losses_step = {k: losses[k](outputs[k], labels[k]) for k in losses}

                running_total_loss.append(sum(list(losses_step.values())))

                for k in output_cat:
                    predictions_categorical[k].append(
                        torch.argmax(torch.softmax(outputs[k], dim=1), 1)
                    )
                    target_categorical[k].append(labels[k])

                for k in output_real:
                    avg_MAE[k].append(losses_step[k])

        for k in predictions_categorical:
            predictions_categorical[k] = torch.cat(predictions_categorical[k])
            target_categorical[k] = torch.cat(target_categorical[k])

        macro_f1_activity = multiclass_f1_score(
            predictions_categorical["Activity"],
            target_categorical["Activity"],
            num_classes=output_cat["Activity"],
            average="macro",
        )

        accuracy = {
            k: multiclass_accuracy(
                predictions_categorical[k],
                target_categorical[k],
                num_classes=output_cat[k],
            )
            for k in output_cat
        }

        avg_MAE = {k: sum(avg_MAE[k]) / len(avg_MAE[k]) for k in avg_MAE}

        val_loss = sum(running_total_loss) / len(running_total_loss)

        print(f"\nVALIDATION")
        for k in accuracy:
            (
                print("{}: acc {:.4f}".format(k, accuracy[k]))
                if k != "Activity"
                else print(
                    "{}: acc {:.4f} macroF1 {:.4f}".format(
                        k, accuracy[k], macro_f1_activity.item()
                    )
                )
            )
        for k in avg_MAE:
            print("{}: MAE {:.4f}".format(k, avg_MAE[k]))
        print("TOTAL_LOSS: {:.4f}".format(val_loss))
        print("epoch time {}s\n".format(time.time() - start_time))

        if epoch == 0:
            best_model = deepcopy(net)
            best_loss = val_loss
        else:
            if val_loss < best_loss:
                best_loss = val_loss
                best_model = deepcopy(net)
                pat_count = 0
                print("new best model found")
            if pat_count == patience:
                print(
                    "Validation performance didn't improve for {} epochs. Training stops.".format(
                        pat_count
                    )
                )
                return best_model
        pat_count += 1

    return best_model

In [24]:
def test_hgnn(net, output_cat, output_real):
    test_loader = DataLoader(X_train, batch_size=128, shuffle=False)
    
    losses = {}
    
    for k in output_cat:
        losses[k] = (
            nn.CrossEntropyLoss()
            if k != "Activity"
            else nn.CrossEntropyLoss(act_weights)
        )
    for k in output_real:
        losses[k] = nn.L1Loss()
    
    
    
    
    
    predictions_categorical = {k: [] for k in output_cat}
    target_categorical = {k: [] for k in output_cat}

    avg_MAE = {k : [] for k in output_real}
    
    total_loss = []
        
    net.eval()
    with torch.no_grad():
        for i, x in enumerate(test_loader):
            x = x.to(device)
            
            labels = x.y
            
            outputs = net(x)
            
     
            losses_step = {k: losses[k](outputs[k], labels[k]).item() for k in losses}
            total_loss.append(sum(list(losses_step.values())))
            
            for k in output_cat:
                    predictions_categorical[k].append(
                        torch.argmax(torch.softmax(outputs[k], dim=1), 1)
                    )
                    target_categorical[k].append(labels[k])
            
            
            for k in output_real:
                    avg_MAE[k].append(losses_step[k])
                    
    for k in predictions_categorical:
            predictions_categorical[k] = torch.cat(predictions_categorical[k])
            target_categorical[k] = torch.cat(target_categorical[k])
               
            
    macro_f1_activity = multiclass_f1_score(
            predictions_categorical["Activity"],
            target_categorical["Activity"],
            num_classes=output_cat["Activity"],
            average="macro",
        )
            
    accuracy = {
            k: multiclass_accuracy(
                predictions_categorical[k],
                target_categorical[k],
                num_classes=output_cat[k],
            )
            for k in output_cat
        }
    
    avg_MAE = {k : sum(avg_MAE[k]) / len(avg_MAE[k]) for k in avg_MAE}
    
    
    Average_total_loss = sum(total_loss) / len(total_loss)
    
    res = {f"{k}_acc" : accuracy[k].item() for k in accuracy} | {"MacroF1Act" : macro_f1_activity.item()} | {f"{k}_mae" : avg_MAE[k] for k in avg_MAE} | {"AVG_total_loss" : Average_total_loss} 
    
    #print(res)
    
    return res

In [ ]:
list_unique = {k : list(tab_all[k].unique()) for k in categorical_columns}
list_unique

In [34]:
outputcat = {k : len(list_unique[k]) if len(list_unique[k]) > 1 else None for k in list_unique}
outputcat = {k : v for k,v in outputcat.items() if v != None}
outputreal = real_value_columns
print(outputcat)
print(outputreal)

{'org:resource': 2, 'Activity': 19, 'org:role': 8, 'case:Project': 79, 'case:Task': 597, 'case:OrganizationalEntity': 36, 'case:Activity': 6, 'case:RfpNumber': 6322}
['time:timestamp', 'case:RequestedAmount']


In [40]:
def train_evaluate(config):
    trained_net = train_hgnn(config, output_cat=outputcat, output_real=outputreal, epochs = 20)
    return test_hgnn(trained_net, output_cat=outputcat, output_real=outputreal)

In [36]:
config = {'lr': 0.0001865322718407359, 'batch_size': 1024, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}

In [37]:
train_evaluate(config)

{'lr': 0.0001865322718407359, 'batch_size': 1024, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}
Epoch: 0




VALIDATION
org:resource: acc 0.5066
Activity: acc 0.0562 macroF1 0.0082
org:role: acc 0.1008
case:Project: acc 0.1537
case:Task: acc 0.0009
case:OrganizationalEntity: acc 0.0903
case:Activity: acc 0.7047
case:RfpNumber: acc 0.0000
time:timestamp: MAE 15.5245
case:RequestedAmount: MAE 2.4729
TOTAL_LOSS: 49.4724
epoch time 14.107543230056763s

Epoch: 1




VALIDATION
org:resource: acc 0.5066
Activity: acc 0.0187 macroF1 0.0038
org:role: acc 0.3723
case:Project: acc 0.1537
case:Task: acc 0.0009
case:OrganizationalEntity: acc 0.0903
case:Activity: acc 0.7047
case:RfpNumber: acc 0.0000
time:timestamp: MAE 14.0935
case:RequestedAmount: MAE 1.5798
TOTAL_LOSS: 45.0189
epoch time 13.19575548171997s

new best model found
Epoch: 2



KeyboardInterrupt: 

In [42]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "hid", "type": "choice", "values": [128], "value_type": "int", "is_ordered" : True,"sort_values":False},
        #{"name": "layers", "type": "choice", "values": [2, 3, 4, 5], "value_type": "int", "is_ordered" : True, "sort_values":False},
        {"name": "layers", "type": "choice", "values": [2], "value_type": "int", "is_ordered" : True, "sort_values":False},
        {"name": "lr", "type": "range", "bounds": [1e-4, 1e-1], "value_type": "float", "log_scale": True},
        {"name": "batch_size", "type": "choice", "values": [128, 256, 512], "value_type": "int", "is_ordered" : True,"sort_values":False}, 
        {"name": "heads", "type": "choice", "values": [1,2], "value_type": "int", "is_ordered" : True,"sort_values":False},
        #{"name": "heads", "type": "choice", "values": [1], "value_type": "int", "is_ordered" : True,"sort_values":False},
        
        #{"name": "aggregation", "type" : "choice", "values" :["sum", "mean", "max"], "value_type" : "str"}
        {"name": "aggregation", "type" : "choice", "values" :["max"], "value_type" : "str"},
     
    ],
  
    evaluation_function=train_evaluate,
    objective_name='AVG_total_loss',
    arms_per_trial=1,
    minimize = True,
    random_seed = 123,
    total_trials = 20
)

print(best_parameters)
means, covariances = values
print(means)
print(experiment)

[INFO 01-09 15:37:29] ax.service.utils.instantiation: Choice parameter hid contains only one value, converting to a fixed parameter instead.
[INFO 01-09 15:37:29] ax.service.utils.instantiation: Choice parameter layers contains only one value, converting to a fixed parameter instead.
[INFO 01-09 15:37:29] ax.service.utils.instantiation: Choice parameter aggregation contains only one value, converting to a fixed parameter instead.
[INFO 01-09 15:37:29] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[FixedParameter(name='hid', parameter_type=INT, value=128), FixedParameter(name='layers', parameter_type=INT, value=2), RangeParameter(name='lr', parameter_type=FLOAT, range=[0.0001, 0.1], log_scale=True), ChoiceParameter(name='batch_size', parameter_type=INT, values=[128, 256, 512], is_ordered=True, sort_values=False), ChoiceParameter(name='heads', parameter_type=INT, values=[1, 2], is_ordered=True, sort_values=False), FixedParameter(name='aggregation', paramete

{'lr': 0.0001865322718407359, 'batch_size': 256, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}
Epoch: 0




VALIDATION
org:resource: acc 0.5066
Activity: acc 0.0039 macroF1 0.0011
org:role: acc 0.5066
case:Project: acc 0.1114
case:Task: acc 0.7074
case:OrganizationalEntity: acc 0.0800
case:Activity: acc 0.7047
case:RfpNumber: acc 0.0000
time:timestamp: MAE 11.3982
case:RequestedAmount: MAE 1.3214
TOTAL_LOSS: 39.3054
epoch time 22.431503534317017s

Epoch: 1




VALIDATION
org:resource: acc 0.5066
Activity: acc 0.0115 macroF1 0.0060
org:role: acc 0.5066
case:Project: acc 0.1114
case:Task: acc 0.7074
case:OrganizationalEntity: acc 0.0794
case:Activity: acc 0.7047
case:RfpNumber: acc 0.0290
time:timestamp: MAE 6.0565
case:RequestedAmount: MAE 1.0618
TOTAL_LOSS: 31.4043
epoch time 23.281733751296997s

new best model found
Epoch: 2




VALIDATION
org:resource: acc 0.5066
Activity: acc 0.2077 macroF1 0.0479
org:role: acc 0.5066
case:Project: acc 0.1114
case:Task: acc 0.7074
case:OrganizationalEntity: acc 0.0800
case:Activity: acc 0.7047
case:RfpNumber: acc 0.0290
time:timestamp: MAE 0.4316
case:RequestedAmount: MAE 1.0198
TOTAL_LOSS: 25.0469
epoch time 22.67598605155945s

new best model found
Epoch: 3



...


VALIDATION
org:resource: acc 0.9970
Activity: acc 0.8394 macroF1 0.5662
org:role: acc 0.9109
case:Project: acc 0.9964
case:Task: acc 0.8345
case:OrganizationalEntity: acc 0.9955
case:Activity: acc 1.0000
case:RfpNumber: acc 0.0012
time:timestamp: MAE 0.3383
case:RequestedAmount: MAE 0.0833
TOTAL_LOSS: 12.0011
epoch time 27.789148569107056s

Epoch: 10




VALIDATION
org:resource: acc 0.9976
Activity: acc 0.8348 macroF1 0.5177
org:role: acc 0.8819
case:Project: acc 0.9964
case:Task: acc 0.8397
case:OrganizationalEntity: acc 0.9949
case:Activity: acc 1.0000
case:RfpNumber: acc 0.0012
time:timestamp: MAE 0.0899
case:RequestedAmount: MAE 0.0823
TOTAL_LOSS: 12.0592
epoch time 27.291767358779907s

Epoch: 11




VALIDATION
org:resource: acc 0.9988
Activity: acc 0.8457 macroF1 0.5159
org:role: acc 0.9055
case:Project: acc 0.9964
case:Task: acc 0.8463
case:OrganizationalEntity: acc 0.9949
case:Activity: acc 1.0000
case:RfpNumber: acc 0.0012
time:timestamp: MAE 0.1456
case:RequestedAmount: MAE 0.1017
TOTAL_LOSS: 12.5536
epoch time 28.631991863250732s

Validation performance didn't improve for 5 epochs. Training stops.


[INFO 01-10 10:55:50] ax.core.experiment: Attached data has some metrics ({'time:timestamp_mae', 'case:Task_acc', 'Activity_acc', 'case:RequestedAmount_mae', 'case:Activity_acc', 'MacroF1Act', 'case:Project_acc', 'org:role_acc', 'org:resource_acc', 'case:OrganizationalEntity_acc', 'case:RfpNumber_acc'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[ERROR 01-10 10:55:50] ax.core.observation: Data contains metric Activity_acc that has not been added to the experiment. You can either update the `optimization_config` or attach it as a tracking metric using `Experiment.add_tracking_metrics` or `AxClient.add_tracking_metrics`. Ignoring all data for metric Activity_acc.
NoneType: None
[ERROR 01-10 10:55:50] ax.core.observation: Data conta

{'lr': 0.027034625220720667, 'batch_size': 128, 'heads': 2, 'hid': 128, 'layers': 2, 'aggregation': 'max'}
{'AVG_total_loss': 9.242907742934355}
Experiment(None)


In [43]:
from ax.service.utils.report_utils import exp_to_df

results = exp_to_df(experiment)

[WARNING 01-10 10:56:12] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


In [47]:
results.sort_values(by="MacroF1Act")

,trial_index,arm_name,trial_status,generation_method,AVG_total_loss,Activity_acc,MacroF1Act,case:Activity_acc,case:OrganizationalEntity_acc,case:Project_acc,case:RequestedAmount_mae,case:RfpNumber_acc,case:Task_acc,org:resource_acc,org:role_acc,time:timestamp_mae,lr,batch_size,heads,hid,layers,aggregation
0,0,0_0,COMPLETED,Sobol,19.483817,0.727930,0.318893,0.998384,0.236095,0.216448,0.014973,0.033084,0.705902,0.843171,0.845042,0.042078,0.000187,256,1,128,2,max
4,4,4_0,COMPLETED,Sobol,16.317488,0.743409,0.439589,0.998384,0.617282,0.493026,0.029920,0.032999,0.705902,0.939786,0.872768,0.085857,0.001014,512,2,128,2,max
15,15,15_0,COMPLETED,BoTorch,9.469370,0.734138,0.478366,0.999660,1.000000,0.999745,0.086785,0.033594,0.862051,0.994812,0.972019,0.155599,0.023720,128,2,128,2,max
16,16,16_0,COMPLETED,BoTorch,10.708646,0.845382,0.508570,0.999405,0.999320,0.996683,0.081061,0.033084,0.794183,0.993366,0.925498,0.227616,0.027035,128,2,128,2,max
19,19,19_0,COMPLETED,BoTorch,8.992352,0.823695,0.528023,0.999660,0.999915,0.999830,0.079420,0.000510,0.938255,0.991665,0.973720,0.128983,0.024116,128,1,128,2,max
11,11,11_0,COMPLETED,BoTorch,9.557756,0.857799,0.533789,1.000000,0.999745,0.998639,0.276674,0.033849,0.878211,0.995152,0.974571,0.283126,0.027816,256,2,128,2,max
1,1,1_0,COMPLETED,Sobol,8.269190,0.819527,0.547508,1.000000,1.000000,0.995663,0.256063,0.038612,0.917418,0.994557,0.991835,0.145315,0.020423,128,2,128,2,max
14,14,14_0,COMPLETED,BoTorch,9.949790,0.816380,0.560061,0.999150,1.000000,1.000000,0.156543,0.035040,0.876340,0.987668,0.927454,0.209423,0.100000,128,1,128,2,max
18,18,18_0,COMPLETED,BoTorch,10.451559,0.884504,0.585030,1.000000,0.999405,0.998469,0.105690,0.000000,0.815360,0.984096,0.979929,0.671028,0.100000,512,1,128,2,max
12,12,12_0,COMPLETED,BoTorch,10.016442,0.790526,0.599229,0.999915,0.999490,0.997704,0.161872,0.025344,0.880507,0.994642,0.925327,0.444710,0.100000,128,2,128,2,max


In [45]:
results = results.sort_values(by="AVG_total_loss")

In [46]:
results.to_csv("BPI20R.csv", sep=",")